# Notebook for Looking at noise comparisons.

Initially written 08 Feb 23 by Craig Lage.

In [ ]:
import sys, os, glob, time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler

In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/bps_13144S"])
exposure=3021120600576

In [ ]:
# Get the eotest results
filename = "/home/c/cslage/u/BOT_LSSTCam/eotest/eotest_gain_13144_15dec21.pkl"
file = open(filename, 'rb')
#fe55_results = pkl.load(file)
ptc_results = pkl.load(file)
file.close()
print(ptc_results.keys())

rafts = [       'R01', 'R02', 'R03', \
         'R10', 'R11', 'R12', 'R13', 'R14', \
         'R20', 'R21', 'R22', 'R23', 'R24', \
         'R30', 'R31', 'R32', 'R33', 'R34', \
                'R41', 'R42', 'R43']
sensors = ['S00', 'S01', 'S02', 'S10', 'S11', 'S12', 'S20', 'S21', 'S22']
print(rafts)
print(ptc_results['ptc_noise']['R01']['S00'])
print(ptc_results['read_noise']['R01']['S00'])

In [ ]:
def detector(raft, sensor):
    # Subroutine to find vendor and detector number given raft and sensor                                                                                                                                                           
    startingCol = [1,0,0,0,1] # First raft column in each row                                                                                                                                                                       
    rows = [0,3,8,13,18] # Starting raft sequence number of each row                                                                                                                                                                
    if raft in ['R11','R12','R13','R14','R21','R22','R23','R24','R30',\
                'R31','R32','R33','R34']:
        vendor = 'E2V'
    else:
        vendor = 'ITL'
    raftRow = int(list(raft)[1])
    raftCol = int(list(raft)[2])
    sensorRow = int(list(sensor)[1])
    sensorCol = int(list(sensor)[2])
    detectorNum = (rows[raftRow] + (raftCol - startingCol[raftRow])) * 9
    detectorNum += 3 * sensorRow + sensorCol
    return vendor, detectorNum, 4 - raftRow, raftCol



In [ ]:
ptc_noise = []
read_noise = []
for RAFT in rafts:
    for SENSOR in sensors:
        VENDOR, DETECTOR, raftRow, raftCol = detector(RAFT, SENSOR)
        if VENDOR == 'ITL':
            ptc_noise += ptc_results['ptc_noise'][RAFT][SENSOR]
            read_noise += ptc_results['read_noise'][RAFT][SENSOR]
plt.scatter(ptc_noise, read_noise)
ptc_fit = []
read_fit = []
min = 5.0
max = 20.0
for i in range(len(ptc_noise)):
    if ptc_noise[i] > min and ptc_noise[i] < max and read_noise[i] > min and read_noise[i] < max:
        ptc_fit.append(ptc_noise[i])
        read_fit.append(read_noise[i])
fit = np.polyfit(ptc_fit, read_fit, 1)
xplot = np.linspace(0,20,200)
yplot = xplot * fit[0] + fit[1]
plt. plot(xplot, yplot, ls='--', color='green')
plt.xlim(0,20)
plt.ylim(0,20)
plt.xlabel("EOTest ptc_noise (ADU)")
plt.ylabel("EOTest read_noise (ADU)")
plt.text(2.5, 17.5, f"Slope = {fit[0]:.2f}, Intercept = {fit[1]:.2f}")

plt.title("EO Test Read noise vs PTC noise")
plt.savefig("/home/c/cslage/u/BOT_LSSTCam/eotest/Noise_Comparison_13144.pdf")